In [ ]:
import re

class Solver:
    
    def __init__(
        self,
        words = '',
        word_file = 'words.txt',
        greens: list[str]=['','','','',''], 
        yellows: dict[str,int]={}, 
        blacks: str = '', 
        forbidden_yellow_pos: list[str]=['','','','','']
    ):
        

        self.greens = greens
        self.yellows = yellows
        self.blacks = blacks
        self.forbidden_yellow_pos = forbidden_yellow_pos
                
        self.words = words
        with open(file=word_file) as f:
            self.words += f.read()
    
    def gen_yellow_range(yellow_char: str):
        chr_num = ord(yellow_char)
        
        if chr_num == 97: #a
            return 'b-z'
        if chr_num == 98: #b
            return 'ac-z'
        if chr_num == 121: #y
            return 'a-xz'
        if chr_num == 122: #z
            return 'a-y'
        
        
        return f'a-{chr(chr_num-1)}{chr(chr_num+1)}-z'
    
    def gen_pattern(self):  
        
        pattern = ''
        for i in range(5):
            if self.greens[i] != '':
                pattern += self.greens[i]
                continue
            cur_block = '[^'
            
            if self.blacks:
                cur_block += self.blacks
            
            if self.forbidden_yellow_pos[i]:
                cur_block += self.forbidden_yellow_pos[i]
            cur_block += ']'
            pattern += cur_block
        pattern = fr'^{pattern}$'
        
        
        for char, cnt in self.yellows.items():
            if char in self.greens: 
                # BIG PROBLEM HERE
                # NEED TO BE ABLE TO RECOGNIZE WHEN A LETTER IS BOTH GREEN AND YELLOW
                # Probably can just work this into the ui
                # Doesn't have to be handled by this function
                # Solution: Just add a letter into yellow and remove it when it becomes green
                cnt += 1
            cur_range = self.gen_yellow_range(char)
            
            # o[a-np-z]*
            # Lookeahead needs to be adjusted to allow for geq cnt
            #   Note: Max amount of duplicate chars in a word is 3
            #   Lol all it took was removing the '$' at the end
            cur_lookahead = fr'(?=^[{cur_range}]*{f"{char}[{cur_range}]*"*cnt})'
            pattern = cur_lookahead + pattern
            
        print(pattern)
        return pattern
    
    def search(self, pattern):
        matches = re.findall(pattern, self.words, re.MULTILINE)
        if not matches:
            print("Not found")
            return None
        return [''.join(match) for match in matches]
    
    def add_guess(self, guess: str, colors: str):
        cur_greens = ''
        if len(guess) != 5 or len(colors) != 5:
            raise ValueError("Guess and colors must be len 5")
        for i, char, color in zip(guess, colors):
            if color == 'b':
                self.blacks += char
                continue
            if color == 'g':
                cur_greens += char
                self.greens[i] = char
                if self.yellows.get(char) is not None:
                    self.yellows[char] -= 1 
                continue